In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [2]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [3]:
test_x = test.iloc[:,:-1]
test_y = test.iloc[:,-1:]
train_x = train.iloc[:,:-1]
train_y = train.iloc[:,-1:]

In [4]:
testX = torch.from_numpy(test_x.values).to(torch.float64)
testY = torch.from_numpy(test_y.values).to(torch.float64)
trainX = torch.from_numpy(train_x.values).to(torch.float64)
trainY = torch.from_numpy(train_y.values).to(torch.float64)

In [5]:
trainY

tensor([[3.],
        [2.],
        [9.],
        ...,
        [5.],
        [2.],
        [5.]], dtype=torch.float64)

In [6]:
#Q1: about how many bytes does trainX consume?
trainX.nelement() * trainX.element_size()

83520

In [7]:
#Q2: what is the biggest difference we would have any one cell if we used float16 instead of float64?
trainX_16 = trainX.to(torch.float16)
trainX_16 = trainX.to(torch.float64)

torch.max(trainX - trainX_16).item()

0.0

In [8]:
#Q3: is a CUDA GPU available on your VM?
torch.cuda.is_available()

False

In [9]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [10]:
#Q4: what is the predicted number of deaths for the first census tract?
(testX[0] @ coef).item()

9.844

In [11]:
#Q5: what is the average number of predicted deaths, over the whole testX dataset?
torch.mean(testX @ coef).item()

12.073632183908048

In [12]:
#Q6: first, what is y when x is a tensor containing 0.0?
x = torch.tensor(0.0)
y = (x**2 - 8*x + 19)
float(y)

19.0

In [13]:
#Q7: what x value minimizes y?
x = torch.tensor(0.0, requires_grad=True)
optimizer = torch.optim.SGD([x],lr =.1)

for epoch in range(25):
    y = ((x**2) - (8*x) + (19))
    y.backward()
    optimizer.step()
    optimizer.zero_grad()
x.item()


3.9848885536193848

In [14]:
#Q8: what is the MSE (mean-square error) when we make predictions using this vector of zero coefficients?
loss_fn = torch.nn.MSELoss()
coef = torch.zeros((10,1),dtype=torch.float64)
preds = trainX @ coef
loss = loss_fn(preds, trainY)
loss.item()

197.8007662835249

In [15]:
torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

In [16]:
loss_fn = torch.nn.MSELoss()

coef = torch.zeros((10, 1), dtype=torch.float64, requires_grad=True)
optimizer = torch.optim.SGD([coef], lr=0.000002)

for epoch in range(500):
    for batchX, batchY in dl:
        predictions = batchX @ coef
        loss = loss_fn(predictions, batchY)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


In [17]:
#Q9: what is the MSE over the training data, using the coefficients resulting from the above training?
X,Y = ds[:]
loss_fn(X @ coef, Y).item()

26.8113940147193

In [18]:
ds = torch.utils.data.TensorDataset(testX, testY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)



In [19]:
#Q10: what is the MSE over the test data?
X,Y = ds[:]
loss_fn(X @ coef, Y).item()

29.05854692548551